In [1]:
import pygmo as pg
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np
import os, sys
main_folder = './../GeneticAlgorithms/'
if main_folder not in sys.path:
    sys.path.append(main_folder)
from utils import ProgressBar

progbar = ProgressBar(elapsed_time=True)

In [2]:
dim=2
algorithm_name = 'AG'
initPop = 300
gen_number = 1000
func_id = 1
output_folder = './results/guedes.joaofelipe/'+algorithm_name
prob = pg.problem(pg.cec2014(prob_id=func_id, dim=dim))

In [3]:
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

In [75]:
def optimize(dim, func_id, initPop=300, perc_maxfes=1, 
             selection='tournament', crossover='exponential', prob_cr=.8, mutation='polynomial', prob_mut=.2, n_trials = 51):

    gen_number = int(10000*dim/initPop)+1
    prob = pg.problem(pg.cec2014(prob_id=func_id, dim=dim))
    shift_data = np.loadtxt("./../GeneticAlgorithms/input_data/shift_data_{}.txt".format(func_id))
    solution = prob.fitness(shift_data[:dim])[0]
    arr_best = list()
    max_fes = perc_maxfes*(10000*dim) 
    max_fes = initPop if max_fes < initPop else max_fes

    for n in range(n_trials):   

        progbar.update_progress(n/float(n_trials))

        genalg = pg.sga(gen=gen_number, # Number of generations
                        cr=prob_cr, # Crossover Probability
        #                elitism=1, # Crossover Probability
                        m=prob_mut, # Mutation Probability       
                        selection=selection, #['tournament', 'truncated', roulette]
                        crossover=crossover, #[single, exponential, binomial, sbx]
                        mutation=mutation, #gaussian, polynomial, uniform
                        eta_c=1 # For sbx crossover [range [1,100]]                          
                        )

        algo = pg.algorithm(genalg)
        algo.set_verbosity(2)

        pop = pg.population(prob, size=initPop, seed=23)
        pop = algo.evolve(pop)    

        uda = algo.extract(pg.core.sga)
        log = uda.get_log()
        hist = pd.DataFrame(log,columns=['generation', 'fevals', 'best', 'improvement'] )
#             print (hist)
        best = hist[hist['fevals'] <= max_fes].tail(1).reset_index(drop=True)['best'][0]
        arr_best.append(best)

    arr_best = np.array(arr_best)
    arr_error = np.abs(arr_best-solution)
    return arr_error, arr_best

In [76]:
arr_dim = [10, 30]
initPop = 300
arr_func_id = [1,2,6,7,9,14]
arr_perc_maxfes = [.001, .01, .1, .2, .3, .4, .5, .6, .7, .8, .9, 1]
selection = 'tournament' # ['tournament', 'truncated', 'roulette']
crossover = 'single' # ['single', 'exponential', 'binomial', 'sbx']
prob_cr = .8 # [.7, .8, .9]
mutation = 'gaussian' # ['gaussian', 'polynomial', 'uniform']
prob_mut = .2 # [.1, .2, .3]
n_trials = 51
for dim in arr_dim:
    print ('> Dim', dim)
    df_results = pd.DataFrame(columns=['function', 'best', 'worst', 'median', 'mean', 'std', 'successRate'])
    for func_id in arr_func_id: 
        print ('\n\n>> F#', func_id)
        df_maxfes = pd.DataFrame(columns=np.append(np.append(['FES_perc'], np.arange(1, n_trials+1, 1)), ['mean']))
        for perc_maxfes in arr_perc_maxfes:
            print ("\tPerc", perc_maxfes)
            arr_error, arr_best = optimize(dim, func_id, initPop, perc_maxfes, 
                                     selection, crossover, prob_cr, mutation, prob_mut, n_trials)
    ###################################


    #################################

            if perc_maxfes == 1:
                df_results.loc[df_results.shape[0]] = ['F'+str(func_id), np.min(arr_error), np.max(arr_error), 
                                                       np.median(arr_error), np.mean(arr_error), np.std(arr_error), 
                                                       arr_best[arr_error < 10e-8].shape[0]/float(n_trials)]


            df_maxfes.loc[df_maxfes.shape[0]] = np.array(list([perc_maxfes]) + list(arr_error) + [np.mean(arr_error)])

        df_maxfes.to_csv(os.path.join(output_folder, '_'.join([algorithm_name, str(dim)])+'.tsv'), sep='\t')
    df_results.to_csv(os.path.join(output_folder, '_'.join([algorithm_name, str(func_id), str(dim)])+'_summarized.tsv'), sep='\t')



>> F# 1
	Perc 0.001
[0:00:14][##########] 98.04% 	Perc 0.01
[0:00:14][##########] 98.04% 	Perc 0.1
[0:00:14][##########] 98.04% 	Perc 0.2
[0:00:14][##########] 98.04% 	Perc 0.3
[0:00:14][##########] 98.04% 	Perc 0.4
[0:00:14][##########] 98.04% 	Perc 0.5
[0:00:14][##########] 98.04% 	Perc 0.6
[0:00:14][##########] 98.04% 	Perc 0.7
[0:00:14][##########] 98.04% 	Perc 0.8
[0:00:14][##########] 98.04% 	Perc 0.9
[0:00:14][##########] 98.04% 	Perc 1
[0:00:14][##########] 98.04% 

>> F# 2
	Perc 0.001
[0:00:11][##########] 98.04% 	Perc 0.01
[0:00:11][##########] 98.04% 	Perc 0.1
[0:00:11][##########] 98.04% 	Perc 0.2
[0:00:11][##########] 98.04% 	Perc 0.3
[0:00:12][##########] 98.04% 	Perc 0.4
[0:00:12][##########] 98.04% 	Perc 0.5
[0:00:11][##########] 98.04% 	Perc 0.6
[0:00:12][##########] 98.04% 	Perc 0.7
[0:00:12][##########] 98.04% 	Perc 0.8
[0:00:12][##########] 98.04% 	Perc 0.9
[0:00:12][##########] 98.04% 	Perc 1
[0:00:12][##########] 98.04% 

>> F# 6
	Perc 0.001
[0:02:58][##########

In [74]:
arr_error

array([1.47547969e+08, 1.47547969e+08])

In [66]:
hist[hist['fevals'] < max_fes]#@.tail(1).reset_index(drop=True)['best'][0]

,generation,fevals,best,improvement


In [77]:
hist

,generation,fevals,best,improvement
0,1,300,1.475481e+08,-9.278066e+06
1,3,900,1.475481e+08,4.374766e+06
2,5,1500,9.891371e+07,-7.962064e+06
3,7,2100,9.839210e+07,4.513219e+07
4,9,2700,5.325991e+07,-1.142751e+07
5,11,3300,5.325991e+07,-6.488077e+06
6,13,3900,3.221409e+07,-1.983520e+07
7,15,4500,3.221409e+07,6.972275e+06
8,17,5100,2.524181e+07,0.000000e+00
9,19,5700,2.524181e+07,-6.972275e+06


In [35]:
df_results

,function,best,worst,median,mean,std,successRate
0,F1,67724.531667,80786.782688,74255.657177,74255.657177,6531.125511,0.0


In [34]:
df_maxfes

,FES_perc,1,2,mean
0,0.001,1.121976e+06,7.513788e+05,936677.284549
1,0.010,3.259531e+05,2.609944e+05,293473.750853
2,0.100,4.024104e+05,6.913152e+05,546862.823016
3,0.200,1.453016e+05,1.439193e+06,792247.304849
4,0.300,1.980046e+05,3.260892e+05,262046.903915
5,0.400,1.687739e+06,2.928478e+05,990293.218276
6,0.500,7.108822e+04,1.770726e+05,124080.395915
7,0.600,1.254848e+06,5.230236e+05,888935.730614
8,0.700,1.699277e+05,1.388441e+06,779184.158446
9,0.800,2.703916e+05,3.067678e+05,288579.687445


In [79]:
print ('sda') if False else None